# 1. Preparación de Entorno

In [4]:
from dotenv import load_dotenv
import os

# Carga las variables de entorno desde el archivo .env
load_dotenv()

bucket = "datasetsbdajac"

#Utilitario para manipular los servicios de AWS
import boto3

In [2]:
access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
region = os.getenv('AWS_DEFAULT_REGION')

# 2. Conexión al servicio

In [3]:
#Obtenemos el cliente de servicio
textract = boto3.client(
  "textract", #Servicio al que nos conectamos
  aws_access_key_id = access_key_id, #Identificador de la clave
  aws_secret_access_key = secret_access_key, #Contraseña de la clave
  region_name = region #Región de la clave
)

# 3. Envío de consulta

In [5]:
#Enviamos la consulta
#Es una consulta asíncrona
respuesta = textract.start_document_text_detection(
    DocumentLocation = { #Indicamos la imagen
        "S3Object": { #Físicamente, la imagen se encuentra en S3
            "Bucket": bucket, #Bucket de la imagen, CAMBIAR "XXX" por nuestras iniciales
            "Name": "imagenes/IMAGEN_1.jpg" #Ruta de la imagen
        }
    }
)

In [6]:
#Obtenemos el Identificador del proceso
idProceso = respuesta["JobId"]

In [7]:
#Verificamos
print(idProceso)

9a9837dab087e134d3b0500c7f4fb09accf54ace48f32e7818682649b778ebc2


# 4. Bucle de espera hasta la finalización del proceso

In [8]:
#Obtenemos el proceso según su identificador
proceso = textract.get_document_text_detection(JobId = idProceso)

In [9]:
#Verificamos el estado del proceso
#Deberemos esperar a que el estado se coloque en "SUCCEEDED" o "FAILED"
print(proceso["JobStatus"])

SUCCEEDED


In [10]:
#Librería para pausar el código
import time

In [11]:
#Entramos en bucle infinito
#TIEMPO: 3 MINUTOS
while True:

  #Obtenemos el proceso según su identificador
  proceso = textract.get_document_text_detection(JobId = idProceso)

  #Verificamos el estado
  if proceso["JobStatus"] in ["SUCCEEDED", "FAILED"]:
    #Indicamos la finalización del proceso
    print("Proceso finalizado!")

    #Si finalizó, salimos del bucle infinito
    break
  else:
    #Mostramos el estado del proceso
    print(proceso["JobStatus"])

  #Detenemos el código por 5 segundos antes de repetir el bucle
  time.sleep(5)

Proceso finalizado!


# 5. Análisis de la respuesta

In [12]:
#Obtenemos el proceso según su identificador
proceso = textract.get_document_text_detection(JobId = idProceso)

In [13]:
#Mostramos el estado de finalización del proceso
print(proceso["JobStatus"])

SUCCEEDED


In [14]:
#En la variable "Blocks" se encuentran los textos
for linea in proceso["Blocks"]:

  #Filtramos sólo las LINE
  if linea["BlockType"] == "LINE":
    print(linea["Text"])

La confluencia entre la novela y la filosofía española de principios del
siglo XX es el hilo conductor de este trabajo, que nace del interés por
mostrar una relectura, en clave hermenéutica, del vigor filosófico pre-
sente en la novelística del momento. Dicho período se inaugura con el
desastre de la Guerra de Cuba, conflicto que propiciará la emergencia
de un grupo de intelectuales imprescindibles en el desarrollo del pen-
samiento español: la generación del 98. Pero el marco contextual per-
tenece a un radio de acción mucho más amplio que tiene que ver con
los movimientos políticos, culturales y sociales que se registran en esa
misma época a nivel europeo. Es decir, no estamos ante una circuns-
tancia aislada en el espacio y el tiempo, sino todo lo contrario: la huida
del solipsismo y del envejecido marco político-cultural hace que los
autores de la generación del 98 asuman las corrientes de pensamiento
que comenzaban su andadura en el resto de Europa.
El proceso al que aludimos, aun

# 6. Modelamiento

In [15]:
#Acumulador de lineas
texto = ""

In [16]:
#Iteramos cada línea
for linea in proceso["Blocks"]:

  #Filtramos sólo las LINE
  if linea["BlockType"] == "LINE":

    #Agregamos la línea y un salto de línea
    texto = texto + linea["Text"] + "\n"

In [17]:
#Verificamos
print(texto)

La confluencia entre la novela y la filosofía española de principios del
siglo XX es el hilo conductor de este trabajo, que nace del interés por
mostrar una relectura, en clave hermenéutica, del vigor filosófico pre-
sente en la novelística del momento. Dicho período se inaugura con el
desastre de la Guerra de Cuba, conflicto que propiciará la emergencia
de un grupo de intelectuales imprescindibles en el desarrollo del pen-
samiento español: la generación del 98. Pero el marco contextual per-
tenece a un radio de acción mucho más amplio que tiene que ver con
los movimientos políticos, culturales y sociales que se registran en esa
misma época a nivel europeo. Es decir, no estamos ante una circuns-
tancia aislada en el espacio y el tiempo, sino todo lo contrario: la huida
del solipsismo y del envejecido marco político-cultural hace que los
autores de la generación del 98 asuman las corrientes de pensamiento
que comenzaban su andadura en el resto de Europa.
El proceso al que aludimos, aun

: 